In [1]:
# # Building Glove

# glove = {}

# with open('glove.6B.100d.txt', 'rb') as f:
#     for l in f:
#         line = str(l)
#         line = line[2:len(line) - 3].split(' ')
#         val = [float(line[i]) for i in range(1, len(line))]
#         glove[line[0]] = val


# matrix_len = len(vocab)
# weights_matrix = np.zeros((matrix_len, 100))
# words_found = 0

# for i, word in enumerate(vocab):
#     try: 
#         weights_matrix[i] = glove[word]
#         words_found += 1
#     except KeyError:
#         weights_matrix[i] = np.random.normal(scale=0.6, size=(100, ))
# weights_matrix = torch.tensor(weights_matrix)

NameError: name 'vocab' is not defined

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import re
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
import re

In [5]:
# # BUILDING

# def create_emb_layer(weights_matrix, non_trainable=False):
#     num_embeddings, embedding_dim = weights_matrix.size()
#     emb_layer = nn.Embedding(num_embeddings, embedding_dim)
#     emb_layer.load_state_dict({'weight': weights_matrix})
#     if non_trainable:
#         emb_layer.weight.requires_grad = False

#     return emb_layer, num_embeddings, embedding_dim

# class LSTMClassifier(nn.Module):
#     def __init__(self, weights_matrix, hidden_size, num_layers, num_classes, dropout = 0.25):
#         super(LSTMClassifier, self).__init__()
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size
#         self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, non_trainable=True)
#         self.LSTM = nn.LSTM(embedding_dim, self.hidden_size, self.num_layers, dropout=dropout, batch_first=True)
#         self.dropout = nn.Dropout(dropout)
#         self.dense = nn.Linear(hidden_size, 256)
#         self.fc = nn.Linear(256, num_classes)

#     def forward(self, x):
#         embed = self.embedding(x)
#         hidden = self.init_hidden(x.size(0))
#         out, _ = self.LSTM(embed, hidden)
#         out = out[:, -1, :]
#         out = self.dropout(out)
#         out = F.relu(self.dense(out))
#         out = self.fc(out)
#         return torch.exp(F.log_softmax(out, dim=1))
    
#     def init_hidden(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_(),
#                       weight.new(self.num_layers, batch_size, self.hidden_size).zero_())
#         return hidden

In [1]:
# ## TRAINING THE LSTM

# vocab_size = len(vocab)
# embedding_dim = 100 # Since we have choosen glove embedding 100 dimenstion version
# hidden_size = 500
# num_classes = 3
# num_epochs = 2
# num_layers = 2
# EPOCHS = 10
# lr=0.001
# criterion = nn.BCELoss()
# clip = 5

# net = LSTMClassifier(weights_matrix, hidden_size, num_layers, num_classes)
# optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# input = torch.randint(1, 4, (5, 3))
# output = net(input)
# net.train()
# for i in range(EPOCHS):
#     counter = 0
#     print('Epoch - '+str(i + 1))
#     for inputs, labels in train_dataloader:
#         counter += 1
#         net.zero_grad()
#         # print(inputs.size())
#         output = net(inputs)
#         loss = criterion(output.squeeze(), labels.float())
#         loss.backward()
#         nn.utils.clip_grad_norm_(net.parameters(), clip)
#         optimizer.step()
        
#         if counter%5 == 0:
#             val_h = net.init_hidden(BATCH_SIZE)
#             val_losses = []
#             net.eval()
#             for inp, lab in valid_dataloader:
#                 # val_h = tuple([each.data for each in val_h])
#                 out = net(inp)
#                 val_loss = criterion(out.squeeze(), lab.float())
#                 val_losses.append(val_loss.item())
                
#             net.train()
#             print("Epoch: {}/{}...".format(i+1, EPOCHS),
#                   "Step: {}...".format(counter),
#                   "Loss: {:.6f}...".format(loss.item()),
#                   "Val Loss: {:.6f}".format(np.mean(val_losses)))
#     print('')

In [4]:
test=['twitter-test1.txt','twitter-test2.txt','twitter-test3.txt']
dev=['twitter-dev-data.txt']
data = {}
tweetids = {}
tweetgts = {}
tweets = {}

for dataset in ['twitter-training-data.txt'] + test + dev:
    data[dataset] = []
    tweets[dataset] = []
    tweetids[dataset] = []
    tweetgts[dataset] = []
    
    with open(dataset, encoding='utf8') as file:
        for check in file:
            senti=check.split('\t')
            tweetids[dataset].append(int(senti[0]))
            
            if (senti[1] == 'positive'):
                tweetgts[dataset].append([0,0,1])
            elif (senti[1] == 'neutral'):
                tweetgts[dataset].append([1,0,0])
            else:
                tweetgts[dataset].append([0,1,0])
            tweets[dataset].append(senti[2])

In [8]:
tokenizer = get_tokenizer('basic_english')
tokens= tokenizer('my name is tushar')

In [13]:
class Words:
    def __init__(self, sentence, oov="<oov>", pad="<pad>"):
        self.sent=sentence
        self.oov=oov
        self.pad=pad
        self.map={}
        self.size=0
        
    def check(self):
        self.map[self.oov]=1
        self.map[self.pad]=0
        self.size=2
        tokenize=[]
        count=2
        for se in self.sent:
            for word in tokenizer(se):
                tokenize.append(word)
        
        for token in tokenize:
            if token not in self.map:
                self.map[token]=count
                count += 1
                self.size += 1
        return self.map
        

In [12]:
tokens

['is', 'my', 'name', 'tushar']